# League of Conservation Voters - Congressional Votes

https://www.lcv.org/congressional-scorecard/congressional-votes/

In [ ]:
import os
import io
import requests
import zipfile
import tempfile
import shutil
import geopandas as gpd
import ibis
from ibis import _
from cng.utils import *
from cng.h3 import *

duckdb_install_h3()
con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
con.raw_sql("SET THREADS=100;")
set_secrets(con)

bucket = "public-election"
s3_prefix = "league_of_conservation_voters/congressional_votes"

# Get all legislative voted on related to conservation

In [ ]:
def process_data(url,save_url):
        columns = ['vote_title','year','roll_call_vote_num','vote_link']

        t= (con.read_csv(url,delim=',', quote='"',header=False,null_padding=True)
            .mutate(row_id=ibis.row_number())
            .rename(vote_title='column0', 
                    year='column1',
                    roll_call_vote_number='column2',
                    vote_link='column3'
                   )
           )
        # weird format, splitting up the df into chambers to organize into 1 edf 
        senate_row = t.filter(_.vote_title =="Senate Votes").select('row_id').limit(1).execute()['row_id'].to_list()[0]
        house_row = t.filter(_.vote_title =="House Votes").select('row_id').limit(1).execute()['row_id'].to_list()[0]
        senate_df = t.filter((_.row_id>senate_row+1)&(_.row_id<house_row)).mutate(chamber=ibis.literal('senate'))
        house_df = t.filter((_.row_id>house_row+1)).mutate(chamber=ibis.literal('house'))
        df=(senate_df.union(house_df)
            .select('year','chamber','vote_title','roll_call_vote_number','vote_link')
            .mutate(votes_for=-1,
                    votes_against=-1,
                    not_voting=-1,
                    pro_enviro_vote=ibis.literal('None'),
                    outcome=ibis.literal('None'),
                   )
            .cast({'roll_call_vote_number':'int64',
                   'votes_for':'int64',
                   'votes_against':'int64',
                   'not_voting':'int64',
                   'pro_enviro_vote':'string',
                   'outcome':'string',
                  })
            .order_by('year','chamber','vote_title')
           )
        #only save if there are any votes
        if df.count().execute()>0:
            df.to_csv(save_url) 
        

def download_csv(bucket,s3_prefix):
    for year in range(1988,2025):
        print(year)
        # year = 1988
        url=f'https://www.lcv.org/congressional-scorecard/congressional-votes/?export=true&show=scorecard&search=conservation&session_year={year}&issue_filter=all&export-type=votes'
        csv_file=f'lcv_congressional_votes_{year}.csv'
        save_url=f's3://{bucket}/{s3_prefix}/{csv_file}'
        process_data(url,save_url)
    # 2025 has to be done separately  
    year = 2025
    csv_file=f'lcv_congressional_votes_{year}.csv'
    save_url=f's3://{bucket}/{s3_prefix}/{csv_file}'
    #different link for most recent year data 
    url=f'https://www.lcv.org/congressional-scorecard/congressional-votes/?export=true&show=recent&search=conservation&issue_filter=all&export-type=votes'
    process_data(url,save_url)
    return 


In [ ]:
download_csv(bucket,s3_prefix)

# Parse through each roll call vote number and collect votes

In [ ]:
from urllib.parse import urlparse

def get_individual_votes(bucket,s3_prefix):
    MAIN_COLUMNS = {
    "year": "bigint",
    "chamber":"varchar",
    "vote_title": "varchar",
    "roll_call_vote_number":"bigint",
    "vote_link":"varchar",
    "votes_for": "bigint",
    "votes_against": "bigint",
    "not_voting": "bigint",
    "pro_enviro_vote": "varchar",
    "outcome": "varchar",
        }
    csv_url = f's3://{bucket}/{s3_prefix}/**.csv'
    temp = con.read_csv(csv_url,columns = MAIN_COLUMNS).filter(_.year>2024).execute()
    print(temp)
    vote_links = temp['vote_link'].to_list()
    years =temp['year'].to_list()
    chambers =temp['chamber'].to_list()
    titles =temp['vote_title'].to_list()

    for year,chamber,title,link in zip(years,chambers,titles,vote_links):
        parsed = urlparse(link)       # parse the URL
        parts  = parsed.path.strip("/").split("/")  # split path by "/"
        slug   = parts[-1]           # last segment
        print(f'{slug}_{year}_{chamber}')
        save_url=f's3://{bucket}/{s3_prefix}/roll_call_votes/{slug}_{year}_{chamber}.csv'
        download_url = f'https://www.lcv.org/roll-call-vote/{slug}/?export=true&sort=state-a-z&export-type=rcv'
        # saving individual legislature into its own csv with tabulated votes  
        t = (con.read_csv(
            download_url,
            skip=8,
            header=True,
            delim=",",
            quote='"',
            escape='"', 
            null_padding=True,
            columns={
                "State":"varchar","First Name":"varchar","Last Name":"varchar","Party":"varchar",
                "District":"varchar","Year Score":"varchar","Lifetime Score":"varchar",
                "Vote Type":"varchar","URL":"varchar",
            })
            .drop("URL")
            .rename(state="State",
                    first_name='First Name',
                    last_name='Last Name',
                    party="Party",
                    district="District",
                    year_score="Year Score",
                    lifetime_score="Lifetime Score",
                    vote_type='Vote Type'
                   )
            .mutate(year=ibis.literal(year))
            .mutate(chamber=ibis.literal(chamber))
            .mutate(vote_title=ibis.literal(title))
            .filter(_.first_name.notnull())
            )
        t.to_csv(save_url)

        # now populate the main table with the vote statistics 
        main_url=f's3://{bucket}/{s3_prefix}/lcv_congressional_votes_{year}.csv'
        pro_votes = int(t.filter(_.vote_type=='pro').count().execute())
        anti_votes = int(t.filter(_.vote_type=='anti').count().execute())
        missed_votes = int(t.filter(_.vote_type=='missed').count().execute())
        a = con.read_csv(download_url)
        col = a.columns 
        s = a.execute()[col[0]].to_list()[2] # recording if voting yay or nay is the pro-enviro vote 
        pro_enviro=s.split(',', 1)[1]
        vote_against = anti_votes if pro_enviro=='Yes' else pro_votes
        vote_for = pro_votes if pro_enviro=='Yes' else anti_votes
        outcome = 'failed' if vote_against>vote_for else 'passed'
        t_new = (con.read_csv(main_url, columns=MAIN_COLUMNS)
            .mutate(
                votes_for=ibis.cases(
                    (_.vote_title == title, ibis.literal(vote_for)),
                    else_=_.votes_for,
                ),
                votes_against=ibis.cases(
                    (_.vote_title == title, ibis.literal(vote_against)),
                    else_=_.votes_against,
                ),
                not_voting=ibis.cases(
                    (_.vote_title == title, ibis.literal(missed_votes)),
                    else_=_.not_voting,
                ),
                pro_enviro_vote=ibis.cases(
                    (_.vote_title == title, ibis.literal(pro_enviro)),
                    else_=_.pro_enviro_vote,
                ),
                outcome=ibis.cases(
                    (_.vote_title == title, ibis.literal(outcome)),
                    else_=_.outcome,
                ),
            )
           .cast({'votes_for':'int64',
                   'votes_against':'int64',
                   'not_voting':'int64',
                   'pro_enviro_vote':'string',
                   'outcome':'string',
                  })
            )
        t_new.to_csv(main_url)


In [ ]:
get_individual_votes(bucket,s3_prefix)